## useful model building.

In [2]:
import tensorflow as tf
slim=tf.contrib.slim

LOGITS_SCOPE_NAME = 'logits'
MERGED_LOGITS_SCOPE = 'merged_logits'
IMAGE_POOLING_SCOPE = 'image_pooling'
ASPP_SCOPE = 'aspp'
CONCAT_PROJECTION_SCOPE = 'concat_projection'
DECODER_SCOPE = 'decoder'

def cal_scaled_dim_val(dim,scale_coeff):
    """利用scale_coeff对dim维做成scale的维度.
       这里只是一个计算,计算scale之后的维度数.其实并没有实际scale数据Tensor对象.
    """
    if isinstance(dim,tf.Tensor):
        return tf.cast((tf.tofloat(dim)-1.0)*scale_coeff+1.0,tf.int32) # 其实这里边的这个+1.0是为了向上取整
    else:
        return (float(dim)-1.0)*scale_coeff+1.0

def _get_logits(images,model_options,weight_decay=0.0001,reuse=None,is_training=False,
               find_tune_batch_norm=False):
    """生成logits网络.该网络应用到aspp,atrous spatial pyramid pooling.
    """
    # 提取features和end_points.
    features,end_points=extract_features(
        images,
        model_options,
        weight_decay=weight_decay,
        reuse=reuse,
        is_training=is_training,
        find_tune_batch_norm=find_tune_batch_norm)
    
    # 如果decoder 有特殊定义的stride.需要对decoder size做scale
    if model_option.decoder_output_stride is not None:
        if model_option.crop_size is None:
            height=tf.shape(images)[1]
            width=tf.shape(images)[2]
        else:
            # crop存在
            height,width=model_option.crop_size
        
        # 求decoder使用的size.这个是经过decoder_output_stride之后的.
        decoder_height=cal_scaled_dim_val(height,1.0/model_options.decoder_output_stride)
        decoder_width=cal_scaled_dim_val(width,1.0/model_options.decoder_output_stride)
        
        # 对features做重新refine
        features=refine_decoder(
            features,
            end_points,
            decoder_height=decoder_height,
            decoder_widht=decoder_width,
            decoder_use_separable_conv=model_options.decoder_use_separable_conv, # 使用离散卷积
            model_variant=model_options.model_variant,
            weight_decay=weight_decay,
            reuse=reuse,
            is_training=is_training,
            fine_tune_batch_norm=find_tune_batch_norm)
        
        # 获得batch的logits
        # batch_normalization作用:
        # 在激活函数之前的bn模块,它接受wx+b计算的feature作为输入.可以做到如下几点:
        # 1. 提高梯度传播数度,将所有输出归一化到0~1.避免梯度消失.
        # 2. 提高模型的收敛速度.(归一化到0~1,所有的feature都是)
        # 3. 减少模型对参数初始化的影响.(归一化到0~1)
        outputs_to_logits={}
        for output in sorted(model_options.outputs_to_num_classes):
            outputs_to_logits[output]=get_batch_logits(
                features,
                model_options.outputs_to_num_classes[output],
                model_options.atrous_rates,
                aspp_with_batch_norm=model_options.aspp_with_batch_norm, # batch normalization
                weight_decay=weight_decay,
                reuse=reuse,
                scope_suffix=output) # scope_suffix后缀
        
        return outputs_to_logits
        

    
def multi_scale_logits(images,
                       model_options,
                       image_pyramid,
                       weight_decay=0.0001,
                       is_training=False,
                       fine_tune_batch_norm=False):
    """构建logits方法
    args:
        model_options: 网络配置的定义信息.
        image_pyramid: 图像金字塔
        weight_decay: 权重衰减
    """
    if not image_pyramid:
        image_pyramid=[1.0] # list
    # crop size
    crop_height=(
        model_options.crop_size[0]
        if model_options.crop_size else tf.shape(images)[1])
    crop_width=(
        model_options.crop_size[1]
        if model_options.crop_size else tf.shape(images)[0])
    # decoder_output_stride 是在decoder单元,提炼分割结果时候使用的 input/output的比
    logits_output_stride=(
        model_options.decoder_output_stride or model_options.output_stride)
    

    logit_height=cal_scaled_dim_val(crop_height,max(1.0,max(image_pyramid))/logits_output_stride) # 这个image_pyramid其实不是很清楚它的含义,是同尺寸的images组还是downsize后images
    logit_width=cal_scaled_dim_val(crop_width,max(1.0,max(image_pyramid))/logits_output_stride)   # 2018-09-17: 这个image_pyramide是包含了一组图像缩放的fractor.并不是图像本身.
    
    outputs_to_scales_to_logits={
        k:{}
        for k in model_options.outputs_to_num_classes
    }
    
    # step 1 对于每一个缩略图
    for image_scale in image_pyramid:
        if image_scale!=1.0:
            # 不是原图,需要缩放
            # 有了缩放因子,需要计算对应的缩放尺寸
            scaled_height=cal_scaled_dim_val(crop_height,image_scale)
            scaled_width=cal_scaled_dim_val(crop_width,image_scale)
            scaled_crop_size=[scaled_height,scaled_width]
            # 有了缩放尺寸,需要对原图做缩放了
            scaled_images=tf.image.resize_bilinear(images,scaled_crop_size,align_corners=True)
            
            if model_options.crops_size:
                scaled_images.set_shape([None,scaled_height,scale_width,3]) # 如果需要crop size的话,我们把scaled_images reshape成3个chn的.
        else:
            # 原图
            scaled_crop_size=model_options.crop_size
            scaled_images=images
        
        # 用做过scale的尺寸替换参数中的crop_size,然后生成网络
        updated_options=model_options._replace(crop_size=scaled_crop_size)
        outputs_to_logits=_get_logits(
            scaled_images,
            updated_options,
            weight_decay=weight_decay,
            reuse=tf.AUTO_REUSE,
            is_training=is_training,
            fine_tune_batch_norm=fine_tune_batch_norm)
        # 此时拿到结果.对结果做一个reshape,以便和其他的scale pyramid做融合使尺寸是合理的.
        for output in sorted(outputs_to_logits):
            outputs_to_logits[output]=tf.image.resize_bilinear(
                outputs_to_logits[output],
                [logit_height,logit_width],
                align_corners=True)
            
        # 只有一层pyramid,就可以返回
        if len(image_pyramid)==1:
            for output in sorted(model_options.outputs_to_num_classes):
                # 第k个scaler fractor对应的LOGITS_SCOPE_NAME,AKA,"logits"
                outputs_to_scales_to_logits[output][LOGITS_SCOPE_NAME]=outputs_to_logits[output]
            
            return outputs_to_scales_to_logits
        
        # 如果有多个pyramid fractor,需要按照对应的标签保存 
        for output in sorted(model_options.outputs_to_num_classes):
            outputs_to_scales_to_logits[output]['logits_%.2f'%image_scale]=outputs_to_logits[output]
            
    # 把多个pyramid fractor融合
    # 需要新创建一个维度,该维度为了融合使用
    for output in model_options.outputs_to_num_classes:
        all_logits=[
            tf.expand_dims(logits,axis=4)
            for logits in outputs_to_scales_to_logits[output].values()
        ]
        # 在这个新维度上做concat( 理解为连接)
        all_logits=tf.concat(all_logits,axis=4)
        # 根据不同的融合方法采用不同的tf的融合方法
        merge_fn=(
            tf.reduce_max
            if model_options.merge_method=='max' else tf.reduce_mean)
        # 在新增维度上融合.
        outputs_to_scales_to_logits[output][MERGED_LOGITS_SCOPE]=merge_fn(all_logits,axis=4)
    
    return outputs_to_scales_to_logits
    
                
    
            
            
    

SyntaxError: invalid syntax (<ipython-input-2-6e9a8b589fb5>, line 20)

 ![image](https://img-blog.csdn.net/20180518234043625?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L29KaU1vRGVZZTEyMzQ1/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
    